# CFO/SRO Synchronization Pipeline (Notebook)

This notebook follows the requested order:

1. **Coarse CFO correction** – remove large frequency offset first so correlation peaks are strong.
2. **SRO estimation & resampling** – estimate sample-rate drift from peak spacing and correct it.
3. **Fine CFO correction** – use phase slope from correlation peaks to remove residual CFO.
4. **Process** – extract CIR/CFR from corrected data.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt


## Cell 1: Read the capture

**Math:** The capture is a discrete-time complex signal \(x[n]\) stored as complex64. We load it into a 1‑D array so downstream correlation and FFTs operate on \(x[n]\).


In [ ]:
# Read the file assuming it's complex floats (fc32)
# Continuous capture (e.g., 5 seconds @ 10 MS/s => 50M samples)
data_path = r"C:\Users\mshif\python\capture_10m.dat"
data = np.fromfile(data_path, dtype=np.complex64)

print(f"Data shape: {data.shape}")


## Cell 2: Define Zadoff–Chu reference

**Math:** The ZC sequence is
\[
  s[n] = e^{-j\pi u n(n+1)/N},\quad n=0,\dots,N-1
\]
It has constant amplitude and ideal cyclic autocorrelation, which gives sharp correlation peaks for synchronization.


In [ ]:
Fs = 10e6            # 10 MS/s (ensure this matches TX/RX)
ZC_ROOT = 1
ZC_LENGTH = 1021      # Match the transmitter sequence length

# Expected spacing between consecutive ZC sequences (samples).
# If you transmit a single ZC back-to-back, expected_spacing = ZC_LENGTH.
# If you insert guard samples, set expected_spacing = ZC_LENGTH + guard_len.
EXPECTED_SPACING = ZC_LENGTH


def generate_zadoff_chu(root, length):
    n = np.arange(length)
    return np.exp(-1j * np.pi * root * n * (n + 1) / length)


ZC_REF = generate_zadoff_chu(ZC_ROOT, ZC_LENGTH).astype(np.complex64)
L = len(ZC_REF)
ZC_FREQ = np.fft.fft(ZC_REF)


## Cell 3: Peak detection utilities

**Math:** We find local maxima of \(|r[k]|\) above a threshold, enforcing a minimum spacing so that peaks represent distinct ZC occurrences.


In [ ]:
def find_correlation_peaks(corr_mag, threshold, min_distance):
    if len(corr_mag) < 3:
        return np.array([], dtype=int)

    candidates = np.where(
        (corr_mag[1:-1] > corr_mag[:-2])
        & (corr_mag[1:-1] > corr_mag[2:])
        & (corr_mag[1:-1] > threshold)
    )[0] + 1

    if len(candidates) == 0:
        return candidates

    selected = [candidates[0]]
    for idx in candidates[1:]:
        if idx - selected[-1] >= min_distance:
            selected.append(idx)
        elif corr_mag[idx] > corr_mag[selected[-1]]:
            selected[-1] = idx

    return np.array(selected, dtype=int)


## Cell 4: Coarse CFO estimation

**Math:** Using two identical halves (Schmidl–Cox), the CFO estimate is
\[
\hat{f}_{	ext{cfo}} = rac{ngle\sum_n y_1[n]y_0^*[n]}{2\pi L}\,F_s.
\]


In [ ]:
def estimate_coarse_cfo(chunk, zc_ref, fs, l):
    corr = np.correlate(chunk, zc_ref, mode="valid")
    peak_idx = int(np.argmax(np.abs(corr)))

    start = peak_idx + len(zc_ref)
    stop = start + 2 * l
    if stop > len(chunk):
        return 0.0

    payload = chunk[start:stop].reshape(2, l)
    z = np.sum(payload[1] * np.conj(payload[0]))
    return (np.angle(z) * fs) / (2 * np.pi * l)


def apply_cfo_correction(x, cfo_hz, fs):
    n = np.arange(len(x))
    rot = np.exp(-1j * 2 * np.pi * cfo_hz * n / fs)
    return x * rot


## Cell 5: SRO estimation and resampling

**Math:** If repeated peaks occur every \(N_0\) samples, a linear fit gives
\[
  	ext{slope} pprox N_0(1+\epsilon),\quad 	ext{SRO(ppm)}=rac{	ext{slope}-N_0}{N_0}\cdot 10^6.
\]
We correct SRO by resampling with factor \((1+\epsilon)\).


In [ ]:
def estimate_sro_from_peaks(peak_indices, expected_spacing):
    if len(peak_indices) < 2:
        return 0.0, expected_spacing

    x = np.arange(len(peak_indices))
    slope, intercept = np.polyfit(x, peak_indices, 1)
    sro_ppm = ((slope - expected_spacing) / expected_spacing) * 1e6
    return sro_ppm, slope


def apply_sro_correction(x, sro_ppm):
    epsilon = sro_ppm * 1e-6
    if abs(epsilon) < 1e-12:
        return x

    n_src = np.arange(len(x))
    n_dst = np.arange(0, int(len(x) / (1.0 + epsilon)))
    idx = n_dst * (1.0 + epsilon)
    real = np.interp(idx, n_src, np.real(x))
    imag = np.interp(idx, n_src, np.imag(x))
    return real + 1j * imag


## Cell 6: Fine CFO from phase slope

**Math:** For correlation peaks \(r[k]\), the phase slope gives residual CFO:
\[
  \phi[k] = 2\pi f_	ext{cfo}	frac{k}{F_s} + \phi_0,\quad
  f_	ext{cfo} = 	frac{	ext{slope} \cdot F_s}{2\pi}.
\]


In [ ]:
def estimate_cfo_from_peak_phases(corr_vals, peak_indices, fs):
    if len(peak_indices) < 2:
        return 0.0, 0.0, 0.0

    phases = np.unwrap(np.angle(corr_vals))
    slope, intercept = np.polyfit(peak_indices, phases, 1)
    cfo_hz = (slope * fs) / (2 * np.pi)
    return cfo_hz, slope, intercept


## Cell 7: Run the pipeline in the requested order

**Math summary:**
1. Coarse CFO correction: \(x'[n] = x[n]e^{-j2\pi f_0 n/F_s}\)
2. SRO correction: resample with factor \((1+\epsilon)\)
3. Fine CFO correction: use phase slope to remove residual CFO
4. Process: extract CIR/CFR using FFT and IFFT


In [ ]:
# 1) Coarse CFO
coarse_chunk = data[:2048]
coarse_cfo = estimate_coarse_cfo(coarse_chunk, ZC_REF, Fs, L)
data_cfo = apply_cfo_correction(data, coarse_cfo, Fs)

# 2) SRO from correlation peaks
corr = np.correlate(data_cfo, ZC_REF, mode="valid")
corr_mag = np.abs(corr)
quiet = np.sort(corr_mag)[: max(1, len(corr_mag) // 4)]
noise_floor = np.mean(quiet)
noise_std = np.std(quiet)
threshold = noise_floor + 6.0 * noise_std
min_distance = max(1, EXPECTED_SPACING // 2)
peak_indices = find_correlation_peaks(corr_mag, threshold, min_distance)

sro_ppm, spacing_slope = estimate_sro_from_peaks(peak_indices, EXPECTED_SPACING)
data_sro = apply_sro_correction(data_cfo, sro_ppm)

# 3) Fine CFO from phase slope
corr_fine = np.correlate(data_sro, ZC_REF, mode="valid")
peak_vals = corr_fine[peak_indices] if len(peak_indices) else np.array([])
fine_cfo, phase_slope, phase_intercept = estimate_cfo_from_peak_phases(
    peak_vals, peak_indices, Fs
)
data_fine = apply_cfo_correction(data_sro, fine_cfo, Fs)

print(f"Coarse CFO: {coarse_cfo:.2f} Hz")
print(f"SRO: {sro_ppm:.3f} ppm")
print(f"Fine CFO: {fine_cfo:.2f} Hz")


## Cell 8: Process peaks to extract CIR/CFR

**Math:** For each symbol \(y[n]\):
\[
  Y[k]=	ext{FFT}(y[n]),\quad H[k]=Y[k]S^*[k],\quad h[n]=	ext{IFFT}(H[k]).
\]


In [ ]:
cfr_mags = []
cir_mags = []

for peak_idx in peak_indices:
    payload_start = peak_idx + len(ZC_REF)
    payload_end = payload_start + L
    if payload_end > len(data_fine):
        break
    sym = data_fine[payload_start:payload_end]
    sym_f = np.fft.fft(sym)
    h_f = sym_f * np.conj(ZC_FREQ)
    h_t = np.fft.ifft(h_f)
    cfr_mags.append(np.abs(h_f))
    cir_mags.append(np.abs(h_t))

print(f"Detected peaks: {len(peak_indices)}")
print(f"Extracted symbols: {len(cfr_mags)}")


## Cell 9: Visualization

**Math:** Magnitude plots show the distribution of CIR/CFR across detected symbols.


In [ ]:
plt.figure(figsize=(12, 10))

plt.subplot(2, 1, 1)
if cir_mags:
    cir_array = np.array(cir_mags).T
    plt.plot(cir_array, color="black", alpha=0.05)
plt.title("CIR Magnitude")
plt.xlabel("Delay (samples)")
plt.ylabel("|h[n]|")
plt.grid(True)

plt.subplot(2, 1, 2)
if cfr_mags:
    cfr_array = np.array(cfr_mags).T
    cfr_db = 20 * np.log10(cfr_array + 1e-12)
    plt.plot(cfr_db, color="black", alpha=0.05)
plt.title("CFR Magnitude (dB)")
plt.xlabel("Subcarrier")
plt.ylabel("Magnitude (dB)")
plt.grid(True)

plt.tight_layout()
plt.show()
